In [128]:
import pandas as pd
from tabulate import tabulate
from decimal import Decimal
from re import sub
import numpy as np
import dateparser

In [141]:
frDf = pd.read_csv("assets_fundraise.csv")
frDf.rename(columns = {
    'Total Raised':'Total Raised ($)',
    'R1 Price':'R1 Price ($)', 
    'R2 Price':'R2 Price ($)',
    'R3 Price':'R3 Price ($)',
    'R4 Price':'R4 Price ($)',
    'R5 Price':'R5 Price ($)',
    'R1 Raise':'R1 Raise ($)', 
    'R2 Raise':'R2 Raise ($)',
    'R3 Raise':'R3 Raise ($)',
    'R4 Raise':'R4 Raise ($)',
    'R5 Raise':'R5 Raise ($)',
}, inplace = True)


for index in frDf.index:
    
    # for each round
    for r in range(1,6):
        
        # normalize price values to USD and remove $
        try:
            frDf.at[index, f"R{r} Price ($)"] = Decimal(sub(r'[^\d.]', '', frDf.at[index, f"R{r} Price ($)"]))
        except:
            pass
        
        # normalize raise values to USD and remove $
        try:
            frDf.at[index, f"R{r} Raise ($)"] = Decimal(sub(r'[^\d.]', '', frDf.at[index, f"R{r} Raise ($)"]))
        except:
            pass

        # normalize date 
        try:
            t = dateparser.parse(frDf.at[index, f"R{r} date"])
            frDf.at[index, f"R{r} date"] = dateparser.parse(frDf.at[index, f"R{r} date"])
        except:
            pass
        

        

/Users/shohihara/Developer/shiftbase/UNCHAIN/data_and_research/messari_dataset/venv/lib/python3.9/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [ ]:
print(tabulate(frDf, headers='keys', tablefmt='grid'))